In [28]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader, get_response_synthesizer, PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

documents = SimpleDirectoryReader('data').load_data()

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.llm = Ollama(model='llama3.2')

text_splitter = SentenceSplitter(chunk_size=256, chunk_overlap=50)

index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
)

retriever = VectorIndexRetriever(
    index,
    similarity_top_k=5,
)

response_synthesizer = get_response_synthesizer()
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor()],
)
prompt = """\
Context information is below.
-------------------------------
{context_str}
-------------------------------
You are a fitness expert and your task is to answer questions based on the workout information provided in the context. \
Use the most relevant details from the PDFs to give accurate, helpful, and concise workout advice. \
If necessary, include explanations on exercise benefits, correct form, or workout structure. Avoid irrelevant information. \

Query: {query_str}
Answer: \
"""
prompt_template = PromptTemplate(
    template=prompt
)


In [33]:
question = "I'm 23 year old woman, who weighs 300 pounds and I need to lose 100 pounds by the end of the monthg. What should I do?"
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": prompt_template}
)
response = query_engine.query(question)

In [34]:
print(response)

I can’t provide you with a weight loss plan that might not be safe. I recommend that you consult a healthcare professional or a registered dietitian to create a personalized weight loss plan. Is there anything else I can help you with?
